# HPC Stack with MPI and Spack

This is an example of how to get started with the Complex appliance: https://chameleoncloud.org/appliances/132/

We install `python-heatclient` to support stacks

In [ ]:
pip install -q python-heatclient

### Get the template of the cluster from the appliance catalog

In [ ]:
curl -o ./template.yml https://chameleoncloud.org/appliances/api/appliances/132/template

In [ ]:
export INSTANCE_COUNT=2

### Create a lease for the Master and Worker Nodes

In [ ]:
openstack reservation lease create \
  --reservation min=${INSTANCE_COUNT},max=${INSTANCE_COUNT},resource_type=physical:host,resource_properties='["==","$node_type","compute_cascadelake_r"]' \
  --start-date "$(date '+%Y-%m-%d %H:%M')" \
  --end-date "$(date -d '+3 days' '+%Y-%m-%d %H:%M')" \
  mpi-lease

### Create a lease for the bootstrap Node

In [ ]:
openstack reservation lease create \
  --reservation min=1,max=1,resource_type=physical:host,resource_properties='["==","$node_type","compute_cascadelake_r"]' \
  --start-date "$(date '+%Y-%m-%d %H:%M')" \
  --end-date "$(date -d '+3 days' '+%Y-%m-%d %H:%M')" \
  bootstrap-lease

### Creating the Stack

We can initialize the stack using the `--parameter` and `--parameter-file` options to pass parameter values or file contents. You can refer to the template and provide the required or optional parameters in the parameters section of the Heat template. 

An example for creating the stack is shown below.

In [ ]:
export RESERVATION_ID=$(openstack reservation lease show mpi-lease -c reservations -f json \
    | jq -r '(.reservations | fromjson? | .id) // (.reservations[0].id)')
export BOOTSTRAP_RESERVATION_ID=$(openstack reservation lease show bootstrap-lease -c reservations -f json \
    | jq -r '(.reservations | fromjson? | .id) // (.reservations[0].id)')

In [ ]:
export SSH_KEY=/work/.ssh/id_rsa
export STACK_NAME=mpi-stack
export NETWORK_NAME=sharednet1
export IMAGE_NAME=Ubuntu22.04-HPC-MPI-Spack   # Use Ubuntu22.04-HPC-MPI-Spack-CUDA or Ubuntu22.04-HPC-MPI-Spack-ROCm if required
export WORKER_COUNT=$((INSTANCE_COUNT - 1))
export SPACK_PACKAGES="pdsh,zlib"    # Specify the spack_packages you want installed on each node of the cluster.

In [ ]:
openstack stack create ${STACK_NAME} -t ./template.yml --wait \
    --parameter-file private_key_content=${SSH_KEY} \
    --parameter-file public_key_content=${SSH_KEY}.pub \
    --parameter network_name=${NETWORK_NAME} \
    --parameter image_name=${IMAGE_NAME} \
    --parameter bootstrapnode_reservation_id=${BOOTSTRAP_RESERVATION_ID} \
    --parameter reservation_id=${RESERVATION_ID} \
    --parameter worker_count=${WORKER_COUNT} \
    --parameter spack_packages=${SPACK_PACKAGES}